# From Image Samples to MIDI Files

This notebook presents an implementation of a function designed to process a compressed file named samples.zip. This compressed file contains a directory consisting of binary images, samples produced by the model. The output of this function is intended to result in a compressed file named midi_samples.zip, containing the MIDI files corresponding to the given samples.

Again, we acknowledge that a significant portion of the development effort invested in this notebook is derived from [here](https://medium.com/analytics-vidhya/convert-midi-file-to-numpy-array-in-python-7d00531890c).

In [ ]:
pip install mido

In [ ]:
import mido
import string
import numpy as np
from PIL import Image
import os
from torchvision import transforms as T,utils

In [ ]:
!unzip samples.zip

## Auxiliary Function

The following function is used to convert a binary array to a MIDI file. The function has been adapted for our purposes. For further details on what exactly it does, interested readers are directed to consult the original work [here](https://medium.com/analytics-vidhya/convert-midi-file-to-numpy-array-in-python-7d00531890c).

In [ ]:
def arry2mid(ary, tempo=500000, step=60):
  # get the difference
  new_ary = np.concatenate([np.array([[0] * 88]), np.array(ary)], axis=0)
  changes = new_ary[1:] - new_ary[:-1]
  # create a midi file with an empty track
  mid_new = mido.MidiFile()
  track = mido.MidiTrack()
  mid_new.tracks.append(track)
  track.append(mido.MetaMessage('set_tempo', tempo=tempo, time=0))
  # add difference in the empty track
  last_time = 0
  for ch in changes:
    if set(ch) == {0}:  # no change
      last_time += step
    else:
      on_notes = np.where(ch > 0)[0]
      on_notes_vol = ch[on_notes]
      off_notes = np.where(ch < 0)[0]
      first_ = True
      for n in off_notes:
        new_time = last_time if first_ else 0
        track.append(mido.Message('note_off', note=n + 21, velocity=0, time=new_time))
        first_ = False
      for n, v in zip(on_notes, on_notes_vol):
        new_time = last_time if first_ else 0
        track.append(mido.Message('note_on', note=n + 21, velocity=60, time=new_time))
        first_ = False

      last_time = step
  return mid_new

## Function ```convert_samples```

The function that converts the binary image samples to MIDI files. The only parameter **step** is used to determine the number of ticks per demisemiquaver - or the thirty-second note. Be aware that this parameter *must* have the same value with the parameter step used to generate the image dataset. Default value is set to 60.

In [ ]:
def convert_samples(step=60):
  transform = T.Compose([T.ToTensor()])
  os.makedirs('midi_samples')
  for image in os.listdir('samples'):
    f = os.path.join('samples', image)
    img = Image.open(f)
    img=transform(img)

    mid_new = arry2mid(ary=img[0],step=step)
    mid_new.save('midi_samples/'+image[0:len(image)-4]+'.mid')

  !zip -r '/content/midi_samples.zip' './midi_samples'